# NBack Frame Extractor

> Extracts frames from n-back videos to facilitate pilot testing.

In [ ]:
#| default_exp nback_frame_extractor

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import cv2
import glob
import shutil
import pandas as pd
import numpy as np
from moviepy.editor import VideoFileClip
import subprocess
from IPython.display import display, HTML

In [ ]:
#| export
class NBackFrameExtractor:
    def __init__(self, input_video_path, frames_folder, trial_images_folder, trial_stimuli_folder, start_time, end_time, resize_factor=1, fps=29.46):
        self.input_video_path = input_video_path
        self.frames_folder = frames_folder
        self.trial_images_folder = trial_images_folder
        self.trial_stimuli_folder = trial_stimuli_folder
        self.start_time = start_time
        self.end_time = end_time
        self.resize_factor = resize_factor
        self.fps = fps
        self.resized_video_path = os.path.join(frames_folder, "resized_video.mp4")
    
    def resize_video_ffmpeg(self):
        print("Resizing video")
        resize_option = f"scale=iw/{self.resize_factor}:ih/{self.resize_factor}"
        command = [
            "ffmpeg",
            "-i", self.input_video_path,
            "-vf", resize_option,
            "-c:a", "copy",  # Copy the audio without re-encoding
            self.resized_video_path
        ]
        subprocess.run(command, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    def extract_frames_ffmpeg(self):
        print("Extracting frames")
        clip = VideoFileClip(self.input_video_path if self.resize_factor == 1 else self.resized_video_path)
        total_duration_seconds = clip.duration

        end_time = self.end_time if self.end_time <= total_duration_seconds else total_duration_seconds
        if not os.path.exists(self.frames_folder):
            os.makedirs(self.frames_folder)
        
        command = [
            "ffmpeg",
            "-i", self.input_video_path if self.resize_factor == 1 else self.resized_video_path,
            "-vf", f"fps={self.fps}",
            "-ss", str(self.start_time),
            "-to", str(end_time),
            "-q:v", "2",  # Set quality for the extracted images
            "-f", "image2",
            os.path.join(self.frames_folder, "frame_%09d.png")
        ]
        
        subprocess.run(command, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        
        frame_number = 1
        current_time = self.start_time
        while current_time < end_time:
            src = os.path.join(self.frames_folder, f"frame_{frame_number:09d}.png")
            dst = os.path.join(self.frames_folder, f"frame_{current_time:.5f}.png")
            if os.path.exists(src):
                os.rename(src, dst)
            current_time += 1 / self.fps
            frame_number += 1

    def analyze_button_color_brightness(self):
        print("Analyzing button colors")
        data = []
        
        for frame_filename in sorted(os.listdir(self.frames_folder)):
            if frame_filename.endswith('.png'):
                frame_path = os.path.join(self.frames_folder, frame_filename)
                frame = cv2.imread(frame_path)
                
                # Define the button region
                button_region_top = 1200
                button_region_bottom = 1300
                button_region_left = 300
                button_region_right = 400
    
                button_region = frame[button_region_top:button_region_bottom, button_region_left:button_region_right]
                
                avg_color_per_row = np.average(button_region, axis=0)
                avg_color = np.average(avg_color_per_row, axis=0)
                avg_brightness = np.average(avg_color)
                
                time = '.'.join(frame_filename.split('_')[1].split('.')[:-1])
                
                data.append([time, avg_color, avg_brightness])
        
        df = pd.DataFrame(data, columns=['time', 'button_color', 'button_brightness'])
        df['yellow'] = df.button_color.apply(lambda x: x[1])
        df['button_pressed'] = df.yellow > 120
        return df



    def detect_trials(self, df):
        print("Detecting trials")
        ref_frame_filename = f"frame_{self.start_time:.5f}.png"
        ref_frame_path = os.path.join(self.frames_folder, ref_frame_filename)
        ref_frame = cv2.imread(ref_frame_path)
        
        # Define the central region from 950 to 970 pixels from the top
        central_region_top = 950
        central_region_bottom = 970
        
        ref_region = ref_frame[central_region_top:central_region_bottom, :]
        
        def calculate_greyness(region):
            # Calculate the standard deviation across the RGB channels
            std_devs = np.std(region, axis=2)
            avg_greyness = np.average(std_devs)
            return avg_greyness
        
        ref_greyness = calculate_greyness(ref_region)
        
        trials = []
        trial_active = False
        trial_start = None
        
        for index, row in df.iterrows():
            frame_filename = f"frame_{row['time']}.png"
            frame_path = os.path.join(self.frames_folder, frame_filename)
            frame = cv2.imread(frame_path)
            
            # Define the same central region for the current frame
            central_region = frame[central_region_top:central_region_bottom, :]
            
            current_greyness = calculate_greyness(central_region)
            
            color_deviation = abs(current_greyness - ref_greyness)
            #print(color_deviation)
            
            if color_deviation < 3.5:  # Adjust threshold as necessary
                if trial_active:
                    trial_active = False
                    trials.append((trial_start, row['time']))
            else:
                if not trial_active:
                    trial_active = True
                    trial_start = row['time']
            
            df.at[index, 'trial_active'] = trial_active
        
        return df.copy(), trials


    def aggregate_trials(self, trial_df):
        aggregated_data = []

        trial_df['time'] = trial_df['time'].astype(float)
        trial_df['trial_active'] = trial_df['trial_active'].astype(int)

        trials = trial_df[trial_df['trial_active'] == 1].groupby((trial_df['trial_active'].shift() != 1).cumsum())
        
        for trial_id, trial_data in trials:
            trial_start_time = trial_data['time'].min()
            trial_end_time = trial_data['time'].max()
            trial_length = trial_end_time - trial_start_time
            avg_button_pressed = trial_data['button_pressed'].mean()
            
            if trial_id > 1:
                previous_trial_end_time = trial_df[(trial_df['trial_active'] == 1) & (trial_df['time'] < trial_start_time)]['time'].max()
                inactive_period = trial_start_time - previous_trial_end_time
            else:
                inactive_period = trial_start_time - trial_df['time'].min()
            
            center_frame_time = trial_start_time + trial_length / 2
            
            button_pressed_times = trial_data[trial_data['button_pressed'] == 1]['time']
            first_button_pressed_time = button_pressed_times.min() if not button_pressed_times.empty else None

            aggregated_data.append({
                'trial_id': trial_id,
                'trial_length': trial_length,
                'avg_button_pressed': avg_button_pressed,
                'inactive_period': inactive_period,
                'center_frame_time': center_frame_time,
                'first_frame_time': trial_start_time,
                'last_frame_time': trial_end_time,
                'first_button_pressed_time': first_button_pressed_time
            })

        aggregated_df = pd.DataFrame(aggregated_data)
        aggregated_df['trial_id'] = aggregated_df.index.values + 1
        return aggregated_df

    def find_closest_frame(self, time, decimal_places=5):
        pattern = os.path.join(self.frames_folder, f"frame_{time:.{decimal_places}f}*.png")
        files = glob.glob(pattern)
        
        if files:
            return files[0]
        else:
            pattern = os.path.join(self.frames_folder, f"frame_*.png")
            all_files = glob.glob(pattern)
            
            closest_file = None
            closest_time_diff = float('inf')
            
            for file in all_files:
                file_time_str = os.path.basename(file).split('_')[1].split('.png')[0]
                try:
                    file_time = float(file_time_str)
                    time_diff = abs(file_time - time)
                    if time_diff < closest_time_diff:
                        closest_time_diff = time_diff
                        closest_file = file
                except ValueError:
                    continue
            
            return closest_file

    def extract_trial_frames(self, aggregated_df, decimal_places=5):
        if not os.path.exists(self.trial_images_folder):
            os.makedirs(self.trial_images_folder)
        
        for index, row in aggregated_df.iterrows():
            trial_id = row['trial_id']
            
            first_frame_path = self.find_closest_frame(row['first_frame_time'], decimal_places)
            #middle_frame_path = self.find_closest_frame(row['center_frame_time'], decimal_places)
            middle_frame_path = self.find_closest_frame(row['first_frame_time']+.6, decimal_places)
            last_frame_path = self.find_closest_frame(row['last_frame_time'], decimal_places)
            
            trial_first_frame_filename = f"trial_{trial_id}_first_frame.png"
            trial_middle_frame_filename = f"trial_{trial_id}_middle_frame.png"
            trial_last_frame_filename = f"trial_{trial_id}_last_frame.png"
            
            for original_path, new_filename in zip(
                [first_frame_path, middle_frame_path, last_frame_path],
                [trial_first_frame_filename, trial_middle_frame_filename, trial_last_frame_filename]
            ):
                if original_path and os.path.exists(original_path):
                    dst_path = os.path.join(self.trial_images_folder, new_filename)
                    shutil.copy(original_path, dst_path)
                else:
                    print(f"File for {new_filename} does not exist")

    def extract_center_stimuli(self, aggregated_df, decimal_places=5):
        if not os.path.exists(self.trial_stimuli_folder):
            os.makedirs(self.trial_stimuli_folder)
        
        for index, row in aggregated_df.iterrows():
            trial_id = row['trial_id']
            
            middle_frame_path = self.find_closest_frame(row['first_frame_time']+.6, decimal_places)
            
            if middle_frame_path and os.path.exists(middle_frame_path):
                img = cv2.imread(middle_frame_path)
                
                # Define the center portion
                y_start = 840
                y_end = 980
                #x_start = 190
                #x_end = 510
                x_start = 0
                x_end = 320
                
                center_portion = img[y_start:y_end, x_start:x_end]
                
                trial_stimuli_filename = f"trial_{trial_id}.png"
                dst_path = os.path.join(self.trial_stimuli_folder, trial_stimuli_filename)
                
                cv2.imwrite(dst_path, center_portion)
            else:
                print(f"Middle frame for trial {trial_id} does not exist")



In [ ]:
#| eval: false
# Convert absolute paths to relative paths
def convert_to_relative_path(path, base_folder):
    return os.path.relpath(path, base_folder)

def path_to_image_html(path):
    return f'<img src="{path}" width="40" height="40"/>'

def analyze_n_back(video_name,start_time,end_time, fps, resize_factor):
    input_video_path = "/Users/hilmarzech/Projects/a02/data/external/pilot_videos/%s"%video_name
    frames_folder = "/Users/hilmarzech/Projects/a02/data/processed/pilot_video_frames/"
    trial_images_folder = "/Users/hilmarzech/Projects/a02/data/processed/trial_images/"
    trial_stimuli_folder = "/Users/hilmarzech/Projects/a02/data/processed/trial_stimuli/"

    
    extractor = NBackFrameExtractor(input_video_path, frames_folder, trial_images_folder, trial_stimuli_folder, start_time, end_time, resize_factor=resize_factor, fps=fps)
    extractor.extract_frames_ffmpeg()
    
    df = extractor.analyze_button_color_brightness()
    trial_df, trials = extractor.detect_trials(df)
    aggregated_df = extractor.aggregate_trials(trial_df)
    aggregated_df['trial_start_time'] = aggregated_df['center_frame_time'] - (aggregated_df['trial_length'] / 2)
    extractor.extract_trial_frames(aggregated_df)
    extractor.extract_center_stimuli(aggregated_df)    
    aggregated_df['button_pressed'] = aggregated_df.first_button_pressed_time.isna()==False
    aggregated_df['stimulus'] = aggregated_df.apply(
        lambda row: convert_to_relative_path(os.path.join(trial_stimuli_folder, f"trial_{row['trial_id']:.1f}.png"), os.getcwd()), axis=1)
#
    return aggregated_df


def display_data(aggregated_df):
    # Add a column with the paths to the center images
    html = aggregated_df.set_index('trial_id')[['stimulus','button_pressed']].to_html(escape=False, formatters={'stimulus': path_to_image_html})
# Display the HTML in the notebook
    display(HTML(html))
    



aggregated_df = analyze_n_back('240605.mp4', 396.5, 556, 55, 1)
#aggregated_df = analyze_n_back('240605.mp4', 511, 525, 55, 1)

display_data(aggregated_df)

#1 back
#start 469 end 788
#2 back 
#start 987 end 1009

Extracting frames
Analyzing button colors
Detecting trials


,stimulus,button_pressed
trial_id,,
1,,False
2,,True
3,,False
4,,False
5,,False
6,,True
7,,False
8,,False
9,,False


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()